In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import PyPDF2
import re

# Read in the Excel file (attached - corpo_announcement) using the pandas' library.
# Extract the relevant columns(like, HEADLINE, NEWSSUB, MORE) containing the earnings call dates, URLs, and any other relevant information.
# (If required info is not found in the text then read the pdf which is present in the "SOURCE" column and extract info from that.)
# Use regular expressions or other methods to identify the links to the earnings call transcripts and audio files within the URL column.
# Output a new Excel file containing the relevant data, along with columns containing the links to the transcripts and audio files.
# Note that you may need to use web scraping techniques to actually download the transcripts and audio files once you have identified their URLs. 
# Be sure to properly handle any errors or exceptions that may occur during the data extraction process.

#1. extract data
#2. filter relevant data (headline, newssub, more, source,ḍttm, newsdt, nsurl,) 
#3. if not found, extract from the pdf

sheet = pd.read_excel('corpo_announcements.xlsx', sheet_name="Sheet1")
columns = ['HEADLINE', 'NEWSSUB', 'DT_TM', 'MORE', 'NEWS_DT', 'NSURL', 'XML_NAME', 'SOURCE']
sheet = sheet[columns]

audiolinks = []

for url in sheet['NSURL']:
    print(url)
    uri = requests.get(url)
    print(uri)
    soup = BeautifulSoup(uri.text, "html.parser")
    corp = soup.find_all('a', attrs={'class':"single-item"})
    for cor in corp:
        if corp.content == " Corp Announcements ":
            tul = corp['href']
            
    uri2 = requests.get(tul)
    soup = BeautifulSoup(uri2.text, "html-parser")
    tables = soup.find('table')
    tabler = tables.find('table')
    tables = tabler.find('table')
    tabler = tables.find_all('table', attrs={'class': "ng-scope"})
    for table in tabler:
        aku = table.find('a')
        if aku is not None:
            tri = str(aku.content)
            tr = "Earnings"
            if tr in tri:
                aki = table.find_next('a')
                if aki is not None:
                    urli = aki['href']
                    urli = str(urli)
                    urli = "https://www.bseindia.com" + urli
                    response = requests.get(urli)
                    with open("doc.pdf", "wb") as file:
                        file.write(response.content)
                    pdfile = open('doc.pdf', 'rb')
                    pdfreader = PyPDF2.PdfFileReader(pdfile)
                    
                    for page in range(pdfreader.getNumPages()):
                        page_obj = pdfreader.getPage(page)
                        page_text = page_obj.extract_text()
                        ulu = re.findall("(?P<url>https?://[^\s]+)", page_text)
                        audiolinks.append(ulu)


sheet['AUDIO_LINK'] = audiolinks

sheet.to_excel('corpo_announcements_extracted.xlsx', index = False)
 
                    
                    
        

ModuleNotFoundError: ignored